## Imports para computer vision

In [ ]:
import time 
import requests
import cv2
import operator
import numpy as np
from __future__ import print_function
import matplotlib.pyplot as plt
%matplotlib inline 

## Url y key del servicio de microsoft

In [ ]:
# Variables
_url = 'https://westus.api.cognitive.microsoft.com/vision/v1.0/analyze'

_key = '5aed9297a1ab46c68d649561213755a6'#None #Here you have to paste your primary key
_maxNumRetries = 10

## Conexión a la base de datos

In [ ]:
from pymongo import MongoClient
client = MongoClient()
# alternatively..
client = MongoClient("localhost",27017)
client = MongoClient("mongodb://localhost:27017/")
db = client["proyecto"]

db.collection_names()


## Obtenemos las colecciones

In [ ]:
collUsMujeres = db['usuarios_mujeres'] 
collUsHombres = db['usuarios_hombres']

collHombres= db.usuarios_hombres.find(no_cursor_timeout=True)
#timeout(False);
print(str(collHombres.count()))

collMujeres= db.usuarios_mujeres.find(no_cursor_timeout=True)
#.timeout(False);

print(str(collMujeres.count()))

collHombresAux = collHombres
collMujeresAux = collMujeres


## Funciones Microsoft

In [ ]:
def processRequest( json, data, headers, params ):

    """
    Helper function to process the request to Project Oxford

    Parameters:
    json: Used when processing images from its URL. See API Documentation
    data: Used when processing image read from disk. See API Documentation
    headers: Used to pass the key information and the data type request
    """

    retries = 0
    result = None

    while True:
        response = requests.request( 'post', _url, json = json, data = data, headers = headers, params = params )
        print("JSON: %s"  % response.json())
        if response.status_code == 429: 

            print( "Message: %s" % ( response.json()['error']['message'] ) )

            if retries <= _maxNumRetries: 
                time.sleep(1) 
                retries += 1
                continue
            else: 
                print( 'Error: failed after retrying!' )
                break

        elif response.status_code == 200 or response.status_code == 201:

            if 'content-length' in response.headers and int(response.headers['content-length']) == 0: 
                result = None 
            elif 'content-type' in response.headers and isinstance(response.headers['content-type'], str): 
                if 'application/json' in response.headers['content-type'].lower(): 
                    result = response.json() if response.content else None 
                elif 'image' in response.headers['content-type'].lower(): 
                    result = response.content
        else:
            print( "Error code: %d" % ( response.status_code ) )
            print( "Message: %s" % ( response.json()['error']['message'] ) )

        break
    
     
    return result

In [ ]:
def renderResultOnImage( result, img ):
    
    """Display the obtained results onto the input image"""

    R = int(result['color']['accentColor'][:2],16)
    G = int(result['color']['accentColor'][2:4],16)
    B = int(result['color']['accentColor'][4:],16)

    cv2.rectangle( img,(0,0), (img.shape[1], img.shape[0]), color = (R,G,B), thickness = 25 )

    if 'categories' in result:
        categoryName = sorted(result['categories'], key=lambda x: x['score'])[0]['name']
        cv2.putText( img, categoryName, (30,70), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,0,0), 3 )

## Obtenemos las imagenes de profile de los usuuarios y llamamos al servicio de microsoft

In [ ]:
import time
import json

tweets = []
erroresInsert = []
cont=0
cantMujeres = collMujeres.count()

#urlImage = 'https://oxfordportal.blob.core.windows.net/vision/Analysis/3.jpg'
urlImage = 'https://scontent.fmvd3-1.fna.fbcdn.net/t31.0-8/13909174_113633669081886_6514259191971925065_o.jpg'

# Computer Vision parameters
params = { 'visualFeatures' : 'Color,Categories,Description,Tags,Faces'} 
data = None

headers = dict()
headers['Ocp-Apim-Subscription-Key'] = _key
headers['Content-Type'] = 'application/json' 

for us in collMujeresAux[0:5]: 
    try: 
        print(us['screen_name'] + " " + str(cont))
        #user = api.get_user(us['screen_name'])
        #urlProfle = user._json['profile_image_url']
        urlProfle=us['profile_image_url']
        urlProfle = urlProfle.replace("_normal", "")
        json = { 'url': urlProfle } 
        
        result = processRequest( json, data, headers, params )
        if result is not None:
            # Load the original image, fetched from the URL
            arr = np.asarray( bytearray( requests.get( urlImage ).content ), dtype=np.uint8 )
            img = cv2.cvtColor( cv2.imdecode( arr, -1 ), cv2.COLOR_BGR2RGB )

            renderResultOnImage( result, img )

            ig, ax = plt.subplots(figsize=(15, 20))
            ax.imshow( img )

        print(user._json['profile_banner_url'])
        print(urlProfle)       
        print("---------------------------------") 
    except Exception as e:
            
            print("us error: " + us['screen_name'])
            print(e)   
    cont = cont +1

   